## 국가별 코로나 확진자 현황 데이터 전처리

- 국가별 코로나 누적 확진자수 현황 시각화
  - 기간 : 2020-01-22 ~ 2022-07-12


- 데이터 시각화 사이트를 활용하여 raw data를 포맷에 맞춰 처리하여 그래프를 만들어보기로 함
  - Flourish : 영상으로 보는 데이터 시각화, 데이터 차트 영상을 쉽게 제작할 수 있는 사이트
  - https://app.flourish.studio 

### 코로나 바이러스 데이터 다운로드

- 국가별 코로나 바이러스 daily 현황 자료
  - Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE) 에서 작성 
  - PDF로 만들어진 공식 문서에서 추출한 데이터와 공식 웹페이지를 크롤링해서 얻은 자료를 CSV 파일로 생성한 것으로 보임
  

- COVID-19-master 폴더 확인
  - 데이터 다운로드: https://github.com/CSSEGISandData/COVID-19
  - 데이터 압축 풀고 COVID-19-master 폴더를 통째로 corona_data 주피터 노트북이 있는 폴더 안에 하위폴더로 이동
  
  

### 데이터 시각화를 위한 데이터 포멧 이해

- 데이터 시각화를 위해, raw data를 변환해야 함
  - 필요 데이터 : 국가명, 국기, 일자별 확진자 수

### raw data 가져오기
- 일부 raw data를 먼저 확인하면서 처리가 필요한 부분을 파악하고 모든 데이터 대상으로 최종 코드 작성

In [3]:
import pandas as pd
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"
doc = pd.read_csv(PATH + "04-01-2020.csv", encoding='utf-8-sig')
doc.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-01 21:58:49,34.223334,-82.461707,4,0,0,4,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-01 21:58:49,30.295065,-92.414197,47,1,0,46,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-01 21:58:49,37.767072,-75.632346,7,0,0,7,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-01 21:58:49,43.452658,-116.241552,195,3,0,192,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-01 21:58:49,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"


In [4]:
import pandas as pd
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"
doc = pd.read_csv(PATH + "03-01-2020.csv", encoding='utf-8-sig')
doc.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-01T10:13:19,66907,2761,31536,30.9756,112.2707
1,NaN,South Korea,2020-03-01T23:43:03,3736,17,30,36.0000,128.0000
2,NaN,Italy,2020-03-01T23:23:02,1694,34,83,43.0000,12.0000
3,Guangdong,Mainland China,2020-03-01T14:13:18,1349,7,1016,23.3417,113.4244
4,Henan,Mainland China,2020-03-01T14:13:18,1272,22,1198,33.8820,113.6140


### 데이터프레임 데이터 변환하기
1. 특정 컬럼만 선택해서 데이터프레임 만들기
   - 3월 중순 데이터까지는 컬럼명이 Province/State, Country/Region 이고, 이후에는 Province_State, Country_Region 이므로  
     try except 구문을 사용해서 데이터 조작
     
     
2. 특정 컬럼에 없는 데이터 삭제하기
   - 확진자수 컬럼(Confirmed)에 결측치가 포함된 데이터 삭제 
   
   
3. 특정 컬럼의 데이터 타입 변경하기
   - 확진자수 컬럼(Confirmed) 데이터 타입을 정수형(int64)으로 변경 

In [5]:
doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
except:
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
    doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
doc = doc.dropna(subset=['Confirmed'])     # 2. 특정 컬럼에 없는 데이터 삭제하기
doc = doc.astype({'Confirmed': 'int64'})   # 3. 특정 컬럼의 데이터 타입 변경하기
doc.head()    

,Province_State,Country_Region,Confirmed
0,Anhui,Mainland China,1
1,Beijing,Mainland China,14
2,Chongqing,Mainland China,6
3,Fujian,Mainland China,1
5,Guangdong,Mainland China,26


- 국가 정보 가져오기

In [6]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig')
country_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,10,AQ,ATA,10.0,NaN,NaN,NaN,Antarctica,-71.94990,23.347000,Antarctica,NaN
3,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
4,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0


- 두 데이터프레임 합쳐보기
   - 확진자 데이터를 모두 포함하고 공통컬럼인 국가명(Country_Region)을 기준으로 데이터 프레임 합치기

In [7]:
test_df = pd.merge(doc, country_info, how='left', on='Country_Region')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3865 entries, 0 to 3864
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Province_State_x  3806 non-null   object 
 1   Country_Region    3865 non-null   object 
 2   Confirmed         3865 non-null   int64  
 3   UID               3839 non-null   float64
 4   iso2              3837 non-null   object 
 5   iso3              3837 non-null   object 
 6   code3             3837 non-null   float64
 7   FIPS              3384 non-null   float64
 8   Admin2            3343 non-null   object 
 9   Province_State_y  3808 non-null   object 
 10  Lat               3698 non-null   float64
 11  Long_             3698 non-null   float64
 12  Combined_Key      3839 non-null   object 
 13  Population        3695 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 452.9+ KB


- 잘못 매칭된 국가 정보 확인하기
  - iso2 컬럼이 매칭되지 않은 확진자수 국가 확인해보기

In [8]:
test_df.isnull().sum()

Province_State_x     59
Country_Region        0
Confirmed             0
UID                  26
iso2                 28
iso3                 28
code3                28
FIPS                481
Admin2              522
Province_State_y     57
Lat                 167
Long_               167
Combined_Key         26
Population          170
dtype: int64

In [9]:
nan_rows = test_df[test_df['iso2'].isnull()]
nan_rows.head()

,Province_State_x,Country_Region,Confirmed,UID,iso2,iso3,code3,FIPS,Admin2,Province_State_y,Lat,Long_,Combined_Key,Population
0,Anhui,Mainland China,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Guangdong,Mainland China,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 국가 컬럼값 변경하기
- Country_Region 국가명이 다양한 경우가 많았음
- 각 케이스를 직접 확인해서, 국가명을 일관되게 변경할 수 있도록 별도 json 파일 작성
- json 파일 기반으로 국가명을 일관되게 변경하기로 함

- 변경할 국가명을 가지고 있는 json 파일 읽기

In [10]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    print (json_data)

{'Mainland China': 'China', 'Macau': 'China', 'South Korea': 'Korea, South', 'Aruba': 'Netherlands', ' Azerbaijan': 'Azerbaijan', 'Bahamas, The': 'Bahamas', 'Cape Verde': 'Cabo Verde', 'Cayman Islands': 'United Kingdom', 'Channel Islands': 'United Kingdom', 'Curacao': 'Netherlands', 'Czech Republic': 'Czechia', 'East Timor': 'Timor-Leste', 'Faroe Islands': 'Denmark', 'French Guiana': 'France', 'Gambia, The': 'Gambia', 'Gibraltar': 'United Kingdom', 'Greenland': 'Denmark', 'Guadeloupe': 'France', 'Guam': 'US', 'Guernsey': 'US', 'Hong Kong': 'China', 'Hong Kong SAR': 'China', 'Iran (Islamic Republic of)': 'Iran', 'Ivory Coast': "Cote d'Ivoire", 'Jersey': 'US', 'Macao SAR': 'China', 'Martinique': 'France', 'Mayotte': 'France', 'North Ireland': 'United Kingdom', 'Palestine': 'West Bank and Gaza', 'Puerto Rico': 'US', 'Republic of Ireland': 'Ireland', 'Republic of Korea': 'Korea, South', 'Republic of Moldova': 'Moldova', 'Republic of the Congo': 'Congo (Brazzaville)', 'Reunion': 'France', '

- 국가명 컬럼(Country_Region) 값을 확인해서 json파일에 존재하는 경우 지정한 국가명으로 변경

In [11]:
def func(row):
    if row['Country_Region'] in json_data:
        row['Country_Region'] = json_data[row['Country_Region']]
    return row

In [12]:
doc = doc.apply(func, axis=1)
doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,China,1
1,Beijing,China,14
2,Chongqing,China,6
3,Fujian,China,1
5,Guangdong,China,26


### 파일명으로 데이터 변환하기

In [13]:
data = "01-22-2020.csv"
date_column = data.split(".")[0].lstrip('0').replace('-', '/')
date_column

'1/22/2020'

In [14]:
doc.columns = ['Province_State', 'Country_Region', date_column]
doc.columns

Index(['Province_State', 'Country_Region', '1/22/2020'], dtype='object')

In [15]:
doc.head()

,Province_State,Country_Region,1/22/2020
0,Anhui,China,1
1,Beijing,China,14
2,Chongqing,China,6
3,Fujian,China,1
5,Guangdong,China,26


### 국가별 총 확진자수 구하기

In [16]:
doc.groupby('Country_Region').sum()

,1/22/2020
Country_Region,
Antarctica,0
China,548
Japan,2
Kiribati,0
"Korea, North",0
"Korea, South",1
Malaysia,0
New Zealand,0
Palau,0


### 데이터 전처리하기
- 지금까지의 과정을 모두 한데 모아서, 함수로 만들기
  1. csv 파일 읽기
  2. 'Country_Region', 'Confirmed' 두 개의 컬럼만 가져오기
  3. 'Confirmed' 에 데이터가 없는 행 삭제하기
  4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
  5. 'Confirmed' 데이터 타입을 int64(정수) 로 변경
  6. 'Country_Region' 를 기준으로 중복된 데이터를 합치기
  7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기

In [17]:
import json
import pandas as pd

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):
    PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
    doc = pd.read_csv(PATH+filename, encoding='utf-8-sig')  # 1. csv 파일 읽기
    try:
        doc = doc[['Country_Region', 'Confirmed']]          # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
    except:
        doc = doc[['Country/Region', 'Confirmed']]
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed'])     # 3. 특정 컬럼에 없는 데이터 삭제하기
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)  # 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
    doc = doc.astype({'Confirmed': 'int64'})   # 5. 특정 컬럼의 데이터 타입 변경하기
    doc = doc.groupby('Country_Region').sum()  # 6. 특정 컬럼으로 중복된 데이터를 합치기
    print(filename)
    # 7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기
    date_column = filename.split(".")[0].lstrip('0').replace('-','/')
    doc.columns = [date_column]
    return doc

In [18]:
doc1 = create_dateframe("01-22-2020.csv")
doc2 = create_dateframe("04-01-2020.csv")
doc2

01-22-2020.csv
04-01-2020.csv


,4/01/2020
Country_Region,
Afghanistan,192
Albania,259
Algeria,847
Andorra,390
Angola,8
...,...
Vietnam,218
West Bank and Gaza,134
Winter Olympics 2022,0


- 데이터 프레임 합치기

In [19]:
doc = pd.merge(doc1, doc2, how='outer', left_index=True, right_index=True)
doc.head()

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,NaN,192
Albania,NaN,259
Algeria,NaN,847
Andorra,NaN,390
Angola,NaN,8


- 없는 데이터는 0값으로 대체하기

In [20]:
doc = doc.fillna(0)
doc.head()

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,0.0,192
Albania,0.0,259
Algeria,0.0,847
Andorra,0.0,390
Angola,0.0,8


- 특정 폴더 파일 리스트 확인하기
  - split() 함수를 사용해서 특정 확장자를 가진 파일 리스트만 추출 가능
  - 문자열변수.split('.') 은 ['파일명', '확장자'] 와 같은 리스트가 반환되므로, 문자열변수.split('.')[-1] 을 통해 
    이 중에서 마지막 아이템을 선택하면 됨
  

In [21]:
import os

PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
file_list = os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)

csv_list

['01-01-2021.csv',
 '01-01-2022.csv',
 '01-02-2021.csv',
 '01-02-2022.csv',
 '01-03-2021.csv',
 '01-03-2022.csv',
 '01-04-2021.csv',
 '01-04-2022.csv',
 '01-05-2021.csv',
 '01-05-2022.csv',
 '01-06-2021.csv',
 '01-06-2022.csv',
 '01-07-2021.csv',
 '01-07-2022.csv',
 '01-08-2021.csv',
 '01-08-2022.csv',
 '01-09-2021.csv',
 '01-09-2022.csv',
 '01-10-2021.csv',
 '01-10-2022.csv',
 '01-11-2021.csv',
 '01-11-2022.csv',
 '01-12-2021.csv',
 '01-12-2022.csv',
 '01-13-2021.csv',
 '01-13-2022.csv',
 '01-14-2021.csv',
 '01-14-2022.csv',
 '01-15-2021.csv',
 '01-15-2022.csv',
 '01-16-2021.csv',
 '01-16-2022.csv',
 '01-17-2021.csv',
 '01-17-2022.csv',
 '01-18-2021.csv',
 '01-18-2022.csv',
 '01-19-2021.csv',
 '01-19-2022.csv',
 '01-20-2021.csv',
 '01-20-2022.csv',
 '01-21-2021.csv',
 '01-21-2022.csv',
 '01-22-2020.csv',
 '01-22-2021.csv',
 '01-22-2022.csv',
 '01-23-2020.csv',
 '01-23-2021.csv',
 '01-23-2022.csv',
 '01-24-2020.csv',
 '01-24-2021.csv',
 '01-24-2022.csv',
 '01-25-2020.csv',
 '01-25-2021

- 파일 리스트 정렬
   - 파일 리스트가 일자 별로 정렬되도록 처리

In [22]:
csv_list_2020 = list()
csv_list_2021 = list()
csv_list_2022 = list()

for name in csv_list:
    print(name)
    csv_year = name.split(".")[0].split("-")[-1]
    if csv_year == '2020':
        csv_list_2020.append(name)
    elif csv_year == '2021': 
        csv_list_2021.append(name)
    else:
        csv_list_2022.append(name)        


01-01-2021.csv
01-01-2022.csv
01-02-2021.csv
01-02-2022.csv
01-03-2021.csv
01-03-2022.csv
01-04-2021.csv
01-04-2022.csv
01-05-2021.csv
01-05-2022.csv
01-06-2021.csv
01-06-2022.csv
01-07-2021.csv
01-07-2022.csv
01-08-2021.csv
01-08-2022.csv
01-09-2021.csv
01-09-2022.csv
01-10-2021.csv
01-10-2022.csv
01-11-2021.csv
01-11-2022.csv
01-12-2021.csv
01-12-2022.csv
01-13-2021.csv
01-13-2022.csv
01-14-2021.csv
01-14-2022.csv
01-15-2021.csv
01-15-2022.csv
01-16-2021.csv
01-16-2022.csv
01-17-2021.csv
01-17-2022.csv
01-18-2021.csv
01-18-2022.csv
01-19-2021.csv
01-19-2022.csv
01-20-2021.csv
01-20-2022.csv
01-21-2021.csv
01-21-2022.csv
01-22-2020.csv
01-22-2021.csv
01-22-2022.csv
01-23-2020.csv
01-23-2021.csv
01-23-2022.csv
01-24-2020.csv
01-24-2021.csv
01-24-2022.csv
01-25-2020.csv
01-25-2021.csv
01-25-2022.csv
01-26-2020.csv
01-26-2021.csv
01-26-2022.csv
01-27-2020.csv
01-27-2021.csv
01-27-2022.csv
01-28-2020.csv
01-28-2021.csv
01-28-2022.csv
01-29-2020.csv
01-29-2021.csv
01-29-2022.csv
01-30-2020

In [23]:
csv_list_2020.sort()
csv_list_2021.sort()
csv_list_2022.sort()
csv_list = csv_list_2020 + csv_list_2021 + csv_list_2022
csv_list

['01-22-2020.csv',
 '01-23-2020.csv',
 '01-24-2020.csv',
 '01-25-2020.csv',
 '01-26-2020.csv',
 '01-27-2020.csv',
 '01-28-2020.csv',
 '01-29-2020.csv',
 '01-30-2020.csv',
 '01-31-2020.csv',
 '02-01-2020.csv',
 '02-02-2020.csv',
 '02-03-2020.csv',
 '02-04-2020.csv',
 '02-05-2020.csv',
 '02-06-2020.csv',
 '02-07-2020.csv',
 '02-08-2020.csv',
 '02-09-2020.csv',
 '02-10-2020.csv',
 '02-11-2020.csv',
 '02-12-2020.csv',
 '02-13-2020.csv',
 '02-14-2020.csv',
 '02-15-2020.csv',
 '02-16-2020.csv',
 '02-17-2020.csv',
 '02-18-2020.csv',
 '02-19-2020.csv',
 '02-20-2020.csv',
 '02-21-2020.csv',
 '02-22-2020.csv',
 '02-23-2020.csv',
 '02-24-2020.csv',
 '02-25-2020.csv',
 '02-26-2020.csv',
 '02-27-2020.csv',
 '02-28-2020.csv',
 '02-29-2020.csv',
 '03-01-2020.csv',
 '03-02-2020.csv',
 '03-03-2020.csv',
 '03-04-2020.csv',
 '03-05-2020.csv',
 '03-06-2020.csv',
 '03-07-2020.csv',
 '03-08-2020.csv',
 '03-09-2020.csv',
 '03-10-2020.csv',
 '03-11-2020.csv',
 '03-12-2020.csv',
 '03-13-2020.csv',
 '03-14-2020

### 여러 데이터 수집, 전처리해서, 하나의 데이터프레임 만들기
- 지금까지의 과정을 모두 한데 모아서, 함수로 만들기
  1. 필요한 파일 리스트만 추출하기
  2. 파일 리스트 정렬하기
  3. 데이터프레임 전처리하기 (별도 create_dateframe() 함수)
  4. 데이터프레임 합치기

In [24]:
import json
import pandas as pd

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):

    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')  # 1. csv 파일 읽기
    try:
        doc = doc[['Country_Region', 'Confirmed']]  # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
    except:
        doc = doc[['Country/Region', 'Confirmed']]  # 2. 특정 컬럼만 선택해서 데이터프레임 만들기
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed'])     # 3. 특정 컬럼에 없는 데이터 삭제하기
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)   # 4. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
    doc = doc.astype({'Confirmed': 'int64'})   # 5. 특정 컬럼의 데이터 타입 변경하기
    doc = doc.groupby('Country_Region').sum()  # 6. 특정 컬럼으로 중복된 데이터를 합치기

    # 7. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기
    date_column = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [date_column]
    return doc

In [25]:
import os

def generate_dateframe_by_path(PATH):

    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split(".")[-1] == 'csv':
            csv_list.append(file)
    
    csv_list_2020 = list()
    csv_list_2021 = list()
    csv_list_2022 = list()

    for name in csv_list:
        csv_year = name.split(".")[0].split("-")[-1]
        if csv_year == '2020':
            csv_list_2020.append(name)
        elif csv_year == '2021': 
            csv_list_2021.append(name)
        else:
            csv_list_2022.append(name)  
    
    csv_list_2020.sort()
    csv_list_2021.sort()
    csv_list_2022.sort()
    csv_list = csv_list_2020 + csv_list_2021 + csv_list_2022
    
    for file in csv_list:
        doc = create_dateframe(file)
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [26]:
PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
doc = generate_dateframe_by_path(PATH)
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,7/03/2022,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,182724.0,182793.0,182793.0,182979.0,183084.0,183221.0,183235.0,183265.0,183268.0,183272.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,282690.0,282690.0,282690.0,283811.0,284758.0,285731.0,286732.0,287984.0,288176.0,289391.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,266128.0,266173.0,266173.0,266181.0,266202.0,266228.0,266246.0,266257.0,266274.0,266303.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44177.0,44177.0,44177.0,44671.0,44671.0,44671.0,44671.0,44671.0,44671.0,44671.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,101320.0,101320.0,101320.0,101320.0,101320.0,101320.0,101320.0,101320.0,101320.0,101320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,659853.0,662490.0,662490.0,662490.0,662490.0,662490.0,662490.0,662490.0,662490.0,662588.0
Winter Olympics 2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,535.0,535.0,535.0,535.0,535.0,535.0,535.0,535.0,535.0,535.0
Yemen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11824.0,11832.0,11832.0,11832.0,11832.0,11832.0,11832.0,11832.0,11832.0,11832.0


In [27]:
doc = doc.astype('int64')
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,7/03/2022,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,182724,182793,182793,182979,183084,183221,183235,183265,183268,183272
Albania,0,0,0,0,0,0,0,0,0,0,...,282690,282690,282690,283811,284758,285731,286732,287984,288176,289391
Algeria,0,0,0,0,0,0,0,0,0,0,...,266128,266173,266173,266181,266202,266228,266246,266257,266274,266303
Andorra,0,0,0,0,0,0,0,0,0,0,...,44177,44177,44177,44671,44671,44671,44671,44671,44671,44671
Angola,0,0,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,101320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,...,659853,662490,662490,662490,662490,662490,662490,662490,662490,662588
Winter Olympics 2022,0,0,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
Yemen,0,0,0,0,0,0,0,0,0,0,...,11824,11832,11832,11832,11832,11832,11832,11832,11832,11832


In [28]:
doc.to_csv("COVID-19-master/final_df.csv")

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석: 1. 데이터의 출처와 주제에 대해 이해</font><br>

- 국가별 코로나 바이러스 daily 현황 자료
- Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE) 에서 작성
- 데이터 소스는 https://github.com/CSSEGISandData/COVID-19/tree/master/who_covid_19_situation_reports 에 명시
  - 실제 데이터 분석에서는 raw data를 어떻게 가져왔는지에 대해서도 세세히 알필요가 있을 때가 있음
    - PDF로 만들어진 공식 문서에서 추출한 데이터와 공식 웹페이지를 크롤링해서 얻은 자료를 CSV 파일로 생성한 것으로 보임
  - 위 데이터는 테스트용 데이터이므로 출처에 대해서는 이 정도만 파악하기로 함
</div>

## 코로나 국가별 확진자 현황 분석/시각화

### 일자별 확진자수 최종 데이터프레임 읽기

In [29]:
import pandas as pd

df_confirmed = pd.read_csv("COVID-19-master/final_df.csv")
df_confirmed.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/03/2022,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,182724,182793,182793,182979,183084,183221,183235,183265,183268,183272
1,Albania,0,0,0,0,0,0,0,0,0,...,282690,282690,282690,283811,284758,285731,286732,287984,288176,289391
2,Algeria,0,0,0,0,0,0,0,0,0,...,266128,266173,266173,266181,266202,266228,266246,266257,266274,266303
3,Andorra,0,0,0,0,0,0,0,0,0,...,44177,44177,44177,44671,44671,44671,44671,44671,44671,44671
4,Angola,0,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,101320


### 국가명과 iso2 매칭 테이블 읽기
  - 국가코드(iso2) 컬럼값으로 http://www.geonames.org 에서 제공하는 국기 이미지 링크를 얻을 수 있음
  - 다음 데이터는 결측치로 변환됨
    - ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’
  - Namibia 국가의 iso2 값이 NA 이므로 결측치 변환을 막기 위해 다음과 같은 옵션 설정
    - keep_default_na=False : 디폴트 결측치 변환 데이터를 사용하지 않고, na_values로 지정한 데이터만 결측치로 변환
    - na_values='' : 결측치로 변환할 값을 지정

In [30]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
country_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,10,AQ,ATA,10.0,NaN,NaN,NaN,Antarctica,-71.94990,23.347000,Antarctica,NaN
3,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
4,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0


In [31]:
country_info[country_info['Country_Region']=='Namibia']

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
445,516,NA,NAM,516.0,NaN,NaN,NaN,Namibia,-22.9576,18.4904,Namibia,2540916.0


- 국가코드(iso2)와 국가명(Country_Region) 컬럼만 가져오기

In [32]:
country_info = country_info[['iso2','Country_Region']]
country_info

,iso2,Country_Region
0,AF,Afghanistan
1,AL,Albania
2,AQ,Antarctica
3,DZ,Algeria
4,AD,Andorra
...,...,...
4312,US,US
4313,US,US
4314,US,US
4315,US,US


- 중복행 제거

In [33]:
country_info.shape

(4317, 2)

In [34]:
country_info = country_info.drop_duplicates(subset='Country_Region', keep='first')
country_info.shape

(200, 2)

In [35]:
country_info

,iso2,Country_Region
0,AF,Afghanistan
1,AL,Albania
2,AQ,Antarctica
3,DZ,Algeria
4,AD,Andorra
...,...,...
849,ZW,Zimbabwe
850,AU,Australia
859,CA,Canada
876,CN,China


### 국가/일자별 확진자수와 국가별 iso2 값 병합

In [36]:
doc_final_country = pd.merge(df_confirmed, country_info, how='left', on = 'Country_Region')
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,182793,182793,182979,183084,183221,183235,183265,183268,183272,AF
1,Albania,0,0,0,0,0,0,0,0,0,...,282690,282690,283811,284758,285731,286732,287984,288176,289391,AL
2,Algeria,0,0,0,0,0,0,0,0,0,...,266173,266173,266181,266202,266228,266246,266257,266274,266303,DZ
3,Andorra,0,0,0,0,0,0,0,0,0,...,44177,44177,44671,44671,44671,44671,44671,44671,44671,AD
4,Angola,0,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,AO


In [37]:
doc_final_country.shape

(199, 905)

### 없는 데이터(NaN) 처리하기

- 없는 데이터(nan) 확인하기

In [38]:
doc_final_country.isnull().sum()

Country_Region    0
1/22/2020         0
1/23/2020         0
1/24/2020         0
1/25/2020         0
                 ..
7/09/2022         0
7/10/2022         0
7/11/2022         0
7/12/2022         0
iso2              7
Length: 905, dtype: int64

In [39]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022,iso2
45,Cruise Ship,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
50,Diamond Princess,0,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,712,712,NaN
108,MS Zaandam,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,NaN
135,Others,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
169,Summer Olympics 2020,0,0,0,0,0,0,0,0,0,...,865,865,865,865,865,865,865,865,865,NaN
174,Taiwan,1,1,3,3,4,5,8,8,9,...,3893643,3929627,3964188,3995621,4026067,4054189,4082028,4101156,4132429,NaN
195,Winter Olympics 2022,0,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,NaN


- 국가명 Taiwan 의 경우 iso2값 매핑되지 않아서 직접 업데이트

In [40]:
doc_final_country.loc[doc_final_country.Country_Region == 'Taiwan', ('iso2')] = 'TW'
doc_final_country[doc_final_country['Country_Region'] == 'Taiwan']

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022,iso2
174,Taiwan,1,1,3,3,4,5,8,8,9,...,3893643,3929627,3964188,3995621,4026067,4054189,4082028,4101156,4132429,TW


- 없는 데이터(nan) 삭제하기

In [41]:
doc_final_country = doc_final_country.dropna(subset=['iso2'])

In [42]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022,iso2


### 국기 링크를 만들어 데이터프레임에 붙이기

- 필요 데이터
  - 국가명, 국기, 일자별 확진자 수
  
  
- 아래 사이트에서 제공하는 국기 이미지를 사용
  - US 는 http://www.geonames.org/flags/x/us.gif

In [43]:
doc_final_country

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,182793,182793,182979,183084,183221,183235,183265,183268,183272,AF
1,Albania,0,0,0,0,0,0,0,0,0,...,282690,282690,283811,284758,285731,286732,287984,288176,289391,AL
2,Algeria,0,0,0,0,0,0,0,0,0,...,266173,266173,266181,266202,266228,266246,266257,266274,266303,DZ
3,Andorra,0,0,0,0,0,0,0,0,0,...,44177,44177,44671,44671,44671,44671,44671,44671,44671,AD
4,Angola,0,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,AO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Vietnam,0,2,2,2,2,2,2,2,2,...,10749324,10750313,10751227,10752140,10752942,10754348,10754813,10755381,10756254,VN
194,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,662490,662490,662490,662490,662490,662490,662490,662490,662588,PS
196,Yemen,0,0,0,0,0,0,0,0,0,...,11832,11832,11832,11832,11832,11832,11832,11832,11832,YE
197,Zambia,0,0,0,0,0,0,0,0,0,...,326259,326259,326408,326651,326794,326794,327052,327102,327102,ZM


In [44]:
pd.set_option('mode.chained_assignment',  None)

def create_flag_link(row):
    row = row.lower()
    flag_link = ' http://www.geonames.org/flags/x/' + row + '.gif'
    return flag_link
    
doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)


In [45]:
doc_final_country

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,182793,182793,182979,183084,183221,183235,183265,183268,183272,http://www.geonames.org/flags/x/af.gif
1,Albania,0,0,0,0,0,0,0,0,0,...,282690,282690,283811,284758,285731,286732,287984,288176,289391,http://www.geonames.org/flags/x/al.gif
2,Algeria,0,0,0,0,0,0,0,0,0,...,266173,266173,266181,266202,266228,266246,266257,266274,266303,http://www.geonames.org/flags/x/dz.gif
3,Andorra,0,0,0,0,0,0,0,0,0,...,44177,44177,44671,44671,44671,44671,44671,44671,44671,http://www.geonames.org/flags/x/ad.gif
4,Angola,0,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,http://www.geonames.org/flags/x/ao.gif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Vietnam,0,2,2,2,2,2,2,2,2,...,10749324,10750313,10751227,10752140,10752942,10754348,10754813,10755381,10756254,http://www.geonames.org/flags/x/vn.gif
194,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,662490,662490,662490,662490,662490,662490,662490,662490,662588,http://www.geonames.org/flags/x/ps.gif
196,Yemen,0,0,0,0,0,0,0,0,0,...,11832,11832,11832,11832,11832,11832,11832,11832,11832,http://www.geonames.org/flags/x/ye.gif
197,Zambia,0,0,0,0,0,0,0,0,0,...,326259,326259,326408,326651,326794,326794,327052,327102,327102,http://www.geonames.org/flags/x/zm.gif


### 데이터프레임 컬럼 조정하기

- 데이터프레임에서 필요한 컬럼만 선택하기

In [46]:
cols = doc_final_country.columns.tolist()
cols

['Country_Region',
 '1/22/2020',
 '1/23/2020',
 '1/24/2020',
 '1/25/2020',
 '1/26/2020',
 '1/27/2020',
 '1/28/2020',
 '1/29/2020',
 '1/30/2020',
 '1/31/2020',
 '2/01/2020',
 '2/02/2020',
 '2/03/2020',
 '2/04/2020',
 '2/05/2020',
 '2/06/2020',
 '2/07/2020',
 '2/08/2020',
 '2/09/2020',
 '2/10/2020',
 '2/11/2020',
 '2/12/2020',
 '2/13/2020',
 '2/14/2020',
 '2/15/2020',
 '2/16/2020',
 '2/17/2020',
 '2/18/2020',
 '2/19/2020',
 '2/20/2020',
 '2/21/2020',
 '2/22/2020',
 '2/23/2020',
 '2/24/2020',
 '2/25/2020',
 '2/26/2020',
 '2/27/2020',
 '2/28/2020',
 '2/29/2020',
 '3/01/2020',
 '3/02/2020',
 '3/03/2020',
 '3/04/2020',
 '3/05/2020',
 '3/06/2020',
 '3/07/2020',
 '3/08/2020',
 '3/09/2020',
 '3/10/2020',
 '3/11/2020',
 '3/12/2020',
 '3/13/2020',
 '3/14/2020',
 '3/15/2020',
 '3/16/2020',
 '3/17/2020',
 '3/18/2020',
 '3/19/2020',
 '3/20/2020',
 '3/21/2020',
 '3/22/2020',
 '3/23/2020',
 '3/24/2020',
 '3/25/2020',
 '3/26/2020',
 '3/27/2020',
 '3/28/2020',
 '3/29/2020',
 '3/30/2020',
 '3/31/2020',
 

- 컬럼 위치 변경

In [47]:
cols.remove('iso2')
cols.insert(1, 'iso2')

In [48]:
doc_final_country = doc_final_country[cols]
doc_final_country

,Country_Region,iso2,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,7/03/2022,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022
0,Afghanistan,http://www.geonames.org/flags/x/af.gif,0,0,0,0,0,0,0,0,...,182724,182793,182793,182979,183084,183221,183235,183265,183268,183272
1,Albania,http://www.geonames.org/flags/x/al.gif,0,0,0,0,0,0,0,0,...,282690,282690,282690,283811,284758,285731,286732,287984,288176,289391
2,Algeria,http://www.geonames.org/flags/x/dz.gif,0,0,0,0,0,0,0,0,...,266128,266173,266173,266181,266202,266228,266246,266257,266274,266303
3,Andorra,http://www.geonames.org/flags/x/ad.gif,0,0,0,0,0,0,0,0,...,44177,44177,44177,44671,44671,44671,44671,44671,44671,44671
4,Angola,http://www.geonames.org/flags/x/ao.gif,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,101320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Vietnam,http://www.geonames.org/flags/x/vn.gif,0,2,2,2,2,2,2,2,...,10748639,10749324,10750313,10751227,10752140,10752942,10754348,10754813,10755381,10756254
194,West Bank and Gaza,http://www.geonames.org/flags/x/ps.gif,0,0,0,0,0,0,0,0,...,659853,662490,662490,662490,662490,662490,662490,662490,662490,662588
196,Yemen,http://www.geonames.org/flags/x/ye.gif,0,0,0,0,0,0,0,0,...,11824,11832,11832,11832,11832,11832,11832,11832,11832,11832
197,Zambia,http://www.geonames.org/flags/x/zm.gif,0,0,0,0,0,0,0,0,...,326259,326259,326259,326408,326651,326794,326794,327052,327102,327102


- 컬럼명 변경

In [49]:
cols[1] = 'Country_Flag'

In [50]:
doc_final_country.columns = cols
doc_final_country.head()

,Country_Region,Country_Flag,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,7/03/2022,7/04/2022,7/05/2022,7/06/2022,7/07/2022,7/08/2022,7/09/2022,7/10/2022,7/11/2022,7/12/2022
0,Afghanistan,http://www.geonames.org/flags/x/af.gif,0,0,0,0,0,0,0,0,...,182724,182793,182793,182979,183084,183221,183235,183265,183268,183272
1,Albania,http://www.geonames.org/flags/x/al.gif,0,0,0,0,0,0,0,0,...,282690,282690,282690,283811,284758,285731,286732,287984,288176,289391
2,Algeria,http://www.geonames.org/flags/x/dz.gif,0,0,0,0,0,0,0,0,...,266128,266173,266173,266181,266202,266228,266246,266257,266274,266303
3,Andorra,http://www.geonames.org/flags/x/ad.gif,0,0,0,0,0,0,0,0,...,44177,44177,44177,44671,44671,44671,44671,44671,44671,44671
4,Angola,http://www.geonames.org/flags/x/ao.gif,0,0,0,0,0,0,0,0,...,101320,101320,101320,101320,101320,101320,101320,101320,101320,101320


### 최종 파일 저장

In [51]:
doc_final_country.to_csv("COVID-19-master/final_covid_data_for_graph.csv")

### 그래프 만들기
- https://app.flourish.studio/ 로그인
- New visualization 선택 -> Bar Chart Race 선택 -> 데이터 수정 (파일 업로드)


- 최종 그래프 : 국가별 코로나 누적 확진자수 
   - https://public.flourish.studio/visualisation/10696763/